# Solution To The Problem Statement 

In [ ]:
#importing the libraries

import urllib
from bs4 import BeautifulSoup, Tag
import pandas as pd
import multiprocessing as mp
from scrape import scrape

In [ ]:
url = "http://www.fallingrain.com" 
page = urllib.request.urlopen(url+"/world/IN/")
soup = BeautifulSoup(page, 'html.parser')           # Extracting the home page into a soup object

In [ ]:
df = pd.DataFrame(columns=['State','City','Latitude','Longitude','Elevation','Estimated Population'] )    # Creating a DataFrame
df.to_csv("Directory_of_Regions_In_India.csv", mode='a', index=False) # For writing the headings to the CSV

In [ ]:
for state in soup.find("ul").contents:  # Since the name of the states were listed under the <ul> tag, iterating through throught the states
    df = pd.DataFrame(columns=['State','City','Latitude','Longitude','Elevation','Estimated Population']) #Initialising Dataframe for new state
    if isinstance(state, Tag):
        name = state.text
        soup = BeautifulSoup(urllib.request.urlopen(url+state.a["href"]), 'html.parser') # Creating a soup object of the link
        
        soup_stack=[soup] # To stack the soup objects of each link      
        
        while True:
            if(len(soup_stack)==0): # To test if extrated all links, by checking empty stack
                break
        
            cur = soup_stack[-1].find(string="Alphabetical listing of Places in "+name) # Bringing the current location to the heading
           
            if cur.find_next(string="Name"):  # To check if it contains the required table 
                for var in cur.find_next("table").contents: 
                    if isinstance(var, Tag):         
                        if var.find(string="Name"):   # To skip the first row of column names
                            continue
                        row = var.find_all(string=True)   # Making a list of the row 
                        if(name=="None"):
                            row.insert(4, "None")    # Since None doesn't have a string under the column Region
                        print(row)                   # Just to check the correct Values while execution

                        # Appending the required values to the dataframe 
                        
                        df=df.append({'State':row[4],'City':row[0],'Latitude':row[8],'Longitude':row[10],'Elevation':row[12],'Estimated Population':row[14]}, ignore_index=True)

                soup_stack.pop() #popping the last object stored

                        
            else: # implies the page does not contain the table this is not the last layer
                soup_stack.pop()  #pop the stack to remove the current link
                
                 
                pool = mp.Pool()  # Creating pool object

                urls = [url+var["href"] for var in cur.find_all_next("a")] #Creating a list of all the links on the page
                results = pool.map(scrape, urls) # Parallelly extracting all the links
                                                 # scrape function in a seperate file, since Jupyter Notebook doesn't allow multiprocessing on functions defined in the same file
                        
                pool.close()
                pool.join()
    
                results=list(reversed(results)) # reversing the contents to ensure correct order while stacking
                for page in results:
                    soup_stack.append(BeautifulSoup(page, 'html.parser')) # stacking all the links

    
    df.to_csv("Directory_of_Regions_In_India.csv", mode='a', header=False, index=False) # Writing to CSV for each state
    

In [ ]:
df